In [ ]:
!pip install torch torchvision matplotlib
!pip install ultralytics
!pip install wandb


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
# Download the augmented dataset
#!wget -O zerowaste-aug.z01 "https://zenodo.org/records/6412647/files/zerowaste-aug.z01?download=1"
#!wget -O zerowaste-aug.z02 "https://zenodo.org/records/6412647/files/zerowaste-aug.z02?download=1"
#!wget -O zerowaste-aug.z03 "https://zenodo.org/records/6412647/files/zerowaste-aug.z03?download=1"
#!wget -O zerowaste-aug.zip "https://zenodo.org/records/6412647/files/zerowaste-aug.zip?download=1"

# Download the final dataset
!wget -O zerowaste-f-final.zip "https://zenodo.org/records/6412647/files/zerowaste-f-final.zip?download=1"

--2024-06-23 13:51:17--  https://zenodo.org/records/6412647/files/zerowaste-f-final.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7518242799 (7.0G) [application/octet-stream]
Saving to: ‘zerowaste-f-final.zip’

zerowaste-f-final.z 100%[===================>]   7.00G  20.9MB/s    in 5m 57s  

2024-06-23 13:57:14 (20.1 MB/s) - ‘zerowaste-f-final.zip’ saved [7518242799/7518242799]



In [ ]:
import zipfile


# Unzip the final dataset
with zipfile.ZipFile("zerowaste-f-final.zip", 'r') as zip_ref:
    zip_ref.extractall("zerowaste-f-final")


In [ ]:
import os
import json
import shutil
import yaml

# Define the base path
base_path = 'zerowaste-f-final/splits_final_deblurred/'

# Paths to the labels JSON files
train_json = os.path.join(base_path, 'train/labels.json')
val_json = os.path.join(base_path, 'val/labels.json')
test_json = os.path.join(base_path, 'test/labels.json')

# Define new dataset structure
train_image_dir = os.path.join(base_path, 'dataset/images/train')
val_image_dir = os.path.join(base_path, 'dataset/images/val')
test_image_dir = os.path.join(base_path, 'dataset/images/test')

train_label_dir = os.path.join(base_path, 'dataset/labels/train')
val_label_dir = os.path.join(base_path, 'dataset/labels/val')
test_label_dir = os.path.join(base_path, 'dataset/labels/test')

# Create new directories if they don't exist
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(test_image_dir, exist_ok=True)

os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)

# Function to convert COCO to YOLO format
def convert_coco_to_yolo(coco_annotation_file, images_dir, labels_output_dir):
    with open(coco_annotation_file) as f:
        coco_data = json.load(f)

    # Create labels directory if it doesn't exist
    os.makedirs(labels_output_dir, exist_ok=True)

    # Create a mapping from category ids to continuous indices starting from 0
    categories = coco_data['categories']
    category_mapping = {category['id']: idx for idx, category in enumerate(categories)}
    class_names = [category['name'] for category in categories]

    # Iterate through images and annotations
    images = {image['id']: image for image in coco_data['images']}
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        image = images[image_id]

        img_width = image['width']
        img_height = image['height']

        # YOLO format: class_id, center_x, center_y, width, height (normalized)
        category_id = annotation['category_id']
        class_id = category_mapping[category_id]

        bbox = annotation['bbox']
        x, y, width, height = bbox
        center_x = (x + width / 2) / img_width
        center_y = (y + height / 2) / img_height
        width /= img_width
        height /= img_height

        # Prepare label file path
        image_filename = os.path.splitext(image['file_name'])[0]
        label_filepath = os.path.join(labels_output_dir, f'{image_filename}.txt')

        # Write annotation to label file
        with open(label_filepath, 'a') as f:
            f.write(f'{class_id} {center_x} {center_y} {width} {height}\n')

        # Move the image to the corresponding directory
        src_image_path = os.path.join(images_dir, image['file_name'])
        dst_image_path = os.path.join(labels_output_dir.replace('labels', 'images'), image['file_name'])
        shutil.copy(src_image_path, dst_image_path)

    return class_names

# Convert the annotations for each dataset split
train_class_names = convert_coco_to_yolo(train_json, os.path.join(base_path, 'train/data'), train_label_dir)
val_class_names = convert_coco_to_yolo(val_json, os.path.join(base_path, 'val/data'), val_label_dir)
test_class_names = convert_coco_to_yolo(test_json, os.path.join(base_path, 'test/data'), test_label_dir)

# Ensure class names are consistent across splits
assert train_class_names == val_class_names == test_class_names
class_names = train_class_names

# Generate the YAML configuration file for YOLOv8
yaml_config = {
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(class_names),
    'names': class_names
}

yaml_file_path = os.path.join(base_path, 'dataset/data.yaml')
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_config, yaml_file, default_flow_style=False)

print(f'YAML configuration file created at {yaml_file_path}')
print('Dataset organized successfully.')


YAML configuration file created at zerowaste-f-final/splits_final_deblurred/dataset/data.yaml
Dataset organized successfully.


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Testing LR impact


In [ ]:
import time
from ultralytics import YOLO

# Define the parameters to test
learning_rates = [0.001, 0.01, 0.1]

# Path to your dataset
dataset_path = '/content/zerowaste-f-final/splits_final_deblurred/dataset/data.yaml'

# Initialize YOLOv8 Nano model
model = YOLO('yolov8n.pt')

# Loop over different learning rates
for lr in learning_rates:
    try:
        print(f"Initializing WandB run for learning rate: {lr}")
        run = wandb.init(project="yolo_training", entity="albin-vernhes", config={"lr0": lr})
        print(f"WandB run initialized for learning rate: {lr}")

        # Train the model and get results
        print(f"Training with learning rate: {lr}")
        results = model.train(data=dataset_path, epochs=3, lr0=lr)
        print(f"Training completed for learning rate: {lr}")

        # Log metrics to wandb for each epoch
        for epoch in range(3):
            # Assuming the results contain a metrics attribute that is a dictionary
            metrics = results

            # Log metrics
            wandb.log({
                "epoch": epoch,
                "learning_rate": lr,
                "train/loss": metrics['train']['loss'][epoch],
                "train/box_loss": metrics['train']['box_loss'][epoch],
                "train/cls_loss": metrics['train']['cls_loss'][epoch],
                "train/dfl_loss": metrics['train']['dfl_loss'][epoch],
                "val/precision": metrics['val']['precision'][epoch],
                "val/recall": metrics['val']['recall'][epoch],
                "val/mAP50": metrics['val']['mAP50'][epoch],
                "val/mAP50-95": metrics['val']['mAP50-95'][epoch],
                "speed/preprocess": metrics['speed']['preprocess'],
                "speed/inference": metrics['speed']['inference'],
                "speed/postprocess": metrics['speed']['postprocess'],
            })
            print(f"Logged metrics for epoch {epoch} and learning rate: {lr}")

        print(f"Finishing WandB run for learning rate: {lr}")
        wandb.finish()
        print(f"WandB run finished for learning rate: {lr}")
    except wandb.errors.CommError as e:
        print(f"Error initializing wandb run: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Ensure wandb is finished to avoid any hanging
        wandb.finish()
        time.sleep(1)  # Give some time for WandB to finalize properly


Initializing WandB run for learning rate: 0.001


WandB run initialized for learning rate: 0.001
Training with learning rate: 0.001
Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/zerowaste-f-final/splits_final_deblurred/dataset/data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /content/zerowaste-f-final/splits_final_deblurred/dataset/labels/train.cache... 2947 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2947/2947 [00:00<?, ?it/s]


...

                   all        571       3687      0.356      0.304      0.269      0.175

3 epochs completed in 0.257 hours.
Optimizer stripped from runs/detect/train422/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train422/weights/best.pt, 6.2MB

Validating runs/detect/train422/weights/best.pt...
Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:41<00:00,  2.29s/it]


                   all        571       3687      0.355      0.304      0.269      0.176
         rigid_plastic        236        324      0.399      0.117      0.202      0.128
             cardboard        543       2361      0.469      0.612      0.515      0.356
                 metal         48         60       0.17     0.0333     0.0263      0.021
          soft_plastic        398        942      0.381      0.453      0.333      0.197
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train422


lr/pg0,▁█▁
lr/pg1,▁█▁
lr/pg2,▁█▁
metrics/mAP50(B),█▁▆
metrics/mAP50-95(B),▆▁█
metrics/precision(B),▁█▅
metrics/recall(B),█▅▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▁▂█


Training completed for learning rate: 0.1
An error occurred: 'DetMetrics' object is not subscriptable


Final with lr = 0.01

In [ ]:
import time
import wandb
from ultralytics import YOLO

# Path to your dataset
dataset_path = '/content/zerowaste-f-final/splits_final_deblurred/dataset/data.yaml'

# Initialize YOLOv8 Nano model
model = YOLO('yolov8n.pt')

try:
    print(f"Initializing WandB run")
    run = wandb.init(project="yolo_training", entity="albin-vernhes")
    print(f"WandB run initialized")

    # Train the model and get results
    print(f"Training")
    results = model.train(data=dataset_path, epochs=30, lr0=0.01)  # Assuming a default learning rate here
    print(f"Training completed")

    # Log metrics to wandb for each epoch
    for epoch in range(10):  # Assuming 30 epochs
        # Accessing metrics correctly
        metrics = results

        # Log metrics
        wandb.log({
            "epoch": epoch,
            "train/loss": metrics['train']['loss'][epoch],
            "train/box_loss": metrics['train']['box_loss'][epoch],
            "train/cls_loss": metrics['train']['cls_loss'][epoch],
            "train/dfl_loss": metrics['train']['dfl_loss'][epoch],
            "val/precision": metrics['val']['precision'][epoch],
            "val/recall": metrics['val']['recall'][epoch],
            "val/mAP50": metrics['val']['mAP50'][epoch],
            "val/mAP50-95": metrics['val']['mAP50-95'][epoch],
            "speed/preprocess": metrics['speed']['preprocess'],
            "speed/inference": metrics['speed']['inference'],
            "speed/postprocess": metrics['speed']['postprocess'],
        })
        print(f"Logged metrics for epoch {epoch}")

    # Save the best model
    model.save('best.pt')  # Adjust this if necessary to the correct path and filename

    print(f"Finishing WandB run")
    wandb.finish()
    print(f"WandB run finished")
except wandb.errors.CommError as e:
    print(f"Error initializing wandb run: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Ensure wandb is finished to avoid any hanging
    wandb.finish()
    time.sleep(1)  # Give some time for WandB to finalize properly


100%|██████████| 6.23M/6.23M [00:00<00:00, 124MB/s]
wandb: Currently logged in as: albin-vernhes. Use `wandb login --relogin` to force relogin


Initializing WandB run


WandB run initialized
Training
Ultralytics YOLOv8.2.40 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/zerowaste-f-final/splits_final_deblurred/dataset/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, s

100%|██████████| 755k/755k [00:00<00:00, 22.7MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        571       3687      0.356       0.34      0.307      0.206

30 epochs completed in 2.243 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.40 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:40<00:00,  2.26s/it]


                   all        571       3687        0.4      0.346      0.328      0.218
         rigid_plastic        236        324      0.434      0.343      0.337      0.226
             cardboard        543       2361      0.544      0.573      0.523      0.357
                 metal         48         60      0.088     0.0333     0.0207      0.015
          soft_plastic        398        942      0.536      0.435      0.432      0.275
Speed: 0.3ms preprocess, 2.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/train


lr/pg0,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg1,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg2,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
metrics/mAP50(B),▁▄▃▃▄▅▃▄▆▄▅▄▆▆▆▆▇▆█▆▆▇▇▇▇▇▇▇▇█
metrics/mAP50-95(B),▁▄▃▂▄▄▃▄▆▄▅▅▆▅▆▆▇▆█▆▆▇▇▇▇▇█▇▇█
metrics/precision(B),▁█▃▃▃▃▂▄▃▂▅▃▄▄▃▄▄▄▅▄▄▄▆▄▄▅▅▅▄▅
metrics/recall(B),▁▄▄▄▃▇▅▅▅▄▅▅▇▅▅▅▇▆█▆▆▇▄██▅▇▅▆█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁


Training completed
An error occurred: 'DetMetrics' object is not subscriptable
